# Evaluation of MSSW on AGRAW1

## Imports

In [1]:
import numpy as np
import pandas as pd
import sklearn

## AGRAW1 dataset locations

In [2]:
abrupt_agraw1_path = '../Datasets_concept_drift/synthetic_data/abrupt_drift/agraw1_1_abrupt_drift_0_noise_balanced.arff'
gradual_agraw1_paths = [
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_05.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_1.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_5.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_10.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw1_1_gradual_drift_0_noise_balanced_20.arff'
]

all_agraw1_data_paths = [abrupt_agraw1_path] + gradual_agraw1_paths

## Accept and preprocess AGRAW1 datasets

In [4]:
from eval_helpers import accepting
from sklearn.preprocessing import MinMaxScaler
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split


agraw1_exclude_reference_batches = {}
agraw1_exclude_testing_batches = {}
agraw1_onehot_reference_batches = {}
agraw1_onehot_testing_batches = {}
agraw1_target_reference_batches = {}
agraw1_target_testing_batches = {}


# agraw1 with categories excluded
for file_path in all_agraw1_data_paths:
    df_x, df_y = accepting.get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = accepting.divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = accepting.divide_numeric_categorical(df_x_test)
    
    reference_data = df_x_ref_num.to_numpy()
    testing_data = df_x_test_num.to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw1_exclude_reference_batches[file_path] = ref_batches
    agraw1_exclude_testing_batches[file_path] = test_batches
    
print('agraw1 exclude')
print(agraw1_exclude_reference_batches)
print(agraw1_exclude_testing_batches)

# agraw1 with categories onehot encoded
for file_path in all_agraw1_data_paths:
    df_x, df_y = accepting.get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = accepting.divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = accepting.divide_numeric_categorical(df_x_test)
    
    ref_index = df_x_ref_cat.index
    test_index = df_x_test_cat.index
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(df_x_ref_cat)
    df_x_ref_cat_transformed = pd.DataFrame(encoder.transform(df_x_ref_cat))
    df_x_test_cat_transformed = pd.DataFrame(encoder.transform(df_x_test_cat))
    df_x_ref_cat_transformed.set_index(ref_index, inplace=True)
    df_x_test_cat_transformed.set_index(test_index, inplace=True)
    
    reference_data = df_x_ref_num.join(df_x_ref_cat_transformed, lsuffix='_num').to_numpy()
    testing_data = df_x_test_num.join(df_x_test_cat_transformed, lsuffix='_num').to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw1_onehot_reference_batches[file_path] = ref_batches
    agraw1_onehot_testing_batches[file_path] = test_batches
    
print('agraw1 onehot')
print(agraw1_onehot_reference_batches)
print(agraw1_onehot_testing_batches)

# agraw1 with categories target encoded
for file_path in all_agraw1_data_paths:
    df_x, df_y = accepting.get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = accepting.divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = accepting.divide_numeric_categorical(df_x_test)
    
    ref_index = df_x_ref_cat.index
    test_index = df_x_test_cat.index
    encoder = TargetEncoder()
    encoder.fit(df_x_ref_cat, df_y_ref)
    df_x_ref_cat_transformed = pd.DataFrame(encoder.transform(df_x_ref_cat))
    df_x_test_cat_transformed = pd.DataFrame(encoder.transform(df_x_test_cat))
    df_x_ref_cat_transformed.set_index(ref_index, inplace=True)
    df_x_test_cat_transformed.set_index(test_index, inplace=True)
    
    reference_data = df_x_ref_num.join(df_x_ref_cat_transformed, lsuffix='_num').to_numpy()
    testing_data = df_x_test_num.join(df_x_test_cat_transformed, lsuffix='_num').to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw1_target_reference_batches[file_path] = ref_batches
    agraw1_target_testing_batches[file_path] = test_batches
    
print('agraw1 target')
print(agraw1_target_reference_batches)
print(agraw1_target_testing_batches)

df               salary    commission   age     elevel       car      zipcode  \
0      115014.164791      0.000000  56.0  b'level3'  b'car15'  b'zipcode5'   
1       39855.706174  20372.573526  73.0  b'level2'   b'car3'  b'zipcode7'   
2       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
3       57795.185560  13344.143894  80.0  b'level2'   b'car1'  b'zipcode3'   
4       46602.621787  26569.114334  46.0  b'level0'  b'car13'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995  142923.630833      0.000000  28.0  b'level0'  b'car13'  b'zipcode3'   
99996   48591.770745  22774.455785  24.0  b'level2'   b'car7'  b'zipcode1'   
99997   40282.441180  70705.062385  24.0  b'level0'  b'car14'  b'zipcode9'   
99998   24701.771322  17773.082776  70.0  b'level2'  b'car14'  b'zipcode1'   
99999  101356.827789      0.000000  63.0  b'level3'  b'car20'  b'zipcode3'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df               salary    commission   age     elevel       car      zipcode  \
0      115014.164791      0.000000  56.0  b'level3'  b'car15'  b'zipcode5'   
1       39855.706174  20372.573526  73.0  b'level2'   b'car3'  b'zipcode7'   
2       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
3       57795.185560  13344.143894  80.0  b'level2'   b'car1'  b'zipcode3'   
4       46602.621787  26569.114334  46.0  b'level0'  b'car13'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   20450.346419  50131.775485  68.0  b'level2'  b'car18'  b'zipcode7'   
99996  142923.630833      0.000000  28.0  b'level0'  b'car13'  b'zipcode3'   
99997   48591.770745  22774.455785  24.0  b'level2'   b'car7'  b'zipcode1'   
99998   40282.441180  70705.062385  24.0  b'level0'  b'car14'  b'zipcode9'   
99999   24701.771322  17773.082776  70.0  b'level2'  b'car14'  b'zipcode1'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df               salary    commission   age     elevel       car      zipcode  \
0      115014.164791      0.000000  56.0  b'level3'  b'car15'  b'zipcode5'   
1       39855.706174  20372.573526  73.0  b'level2'   b'car3'  b'zipcode7'   
2       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
3       57795.185560  13344.143894  80.0  b'level2'   b'car1'  b'zipcode3'   
4       46602.621787  26569.114334  46.0  b'level0'  b'car13'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   82084.560570      0.000000  38.0  b'level1'   b'car8'  b'zipcode1'   
99996  122185.589049      0.000000  68.0  b'level2'  b'car16'  b'zipcode5'   
99997   75112.189101      0.000000  43.0  b'level0'  b'car14'  b'zipcode3'   
99998   77645.133039      0.000000  78.0  b'level4'   b'car9'  b'zipcode6'   
99999   20450.346419  50131.775485  68.0  b'level2'  b'car18'  b'zipcode7'   

             hvalue  hyears           loan      class  
0   

...

df               salary    commission   age     elevel       car      zipcode  \
0      115014.164791      0.000000  56.0  b'level3'  b'car15'  b'zipcode5'   
1       39855.706174  20372.573526  73.0  b'level2'   b'car3'  b'zipcode7'   
2       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
3       57795.185560  13344.143894  80.0  b'level2'   b'car1'  b'zipcode3'   
4       46602.621787  26569.114334  46.0  b'level0'  b'car13'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995  129778.800352      0.000000  37.0  b'level2'   b'car1'  b'zipcode8'   
99996  124575.146339      0.000000  53.0  b'level0'   b'car7'  b'zipcode4'   
99997   65021.355936  43217.169171  39.0  b'level3'  b'car18'  b'zipcode4'   
99998  113209.808491      0.000000  53.0  b'level2'  b'car15'  b'zipcode2'   
99999   60121.727315  24438.661331  62.0  b'level1'  b'car14'  b'zipcode2'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issue

df               salary    commission   age     elevel       car      zipcode  \
0      115014.164791      0.000000  56.0  b'level3'  b'car15'  b'zipcode5'   
1       39855.706174  20372.573526  73.0  b'level2'   b'car3'  b'zipcode7'   
2       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
3       57795.185560  13344.143894  80.0  b'level2'   b'car1'  b'zipcode3'   
4       46602.621787  26569.114334  46.0  b'level0'  b'car13'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   37724.568475  11570.033669  27.0  b'level1'   b'car8'  b'zipcode3'   
99996  110548.635896      0.000000  58.0  b'level4'  b'car20'  b'zipcode2'   
99997  119805.259163      0.000000  73.0  b'level1'  b'car15'  b'zipcode1'   
99998  120012.609799      0.000000  48.0  b'level1'   b'car9'  b'zipcode7'   
99999  120413.864305      0.000000  38.0  b'level4'   b'car9'  b'zipcode3'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issue

agraw1 target
{'../Datasets_concept_drift/synthetic_data/abrupt_drift/agraw1_1_abrupt_drift_0_noise_balanced.arff': [array([[1.15014165e+05, 0.00000000e+00, 5.60000000e+01, ...,
        4.98438784e-01, 5.19359146e-01, 5.03714710e-01],
       [3.98557062e+04, 2.03725735e+04, 7.30000000e+01, ...,
        4.96711608e-01, 4.96388707e-01, 4.96196606e-01],
       [8.77012909e+04, 0.00000000e+00, 5.20000000e+01, ...,
        4.94607347e-01, 4.78290366e-01, 5.03714710e-01],
       ...,
       [1.23384354e+05, 0.00000000e+00, 6.20000000e+01, ...,
        4.94607347e-01, 5.26588846e-01, 4.96196606e-01],
       [1.00628871e+05, 0.00000000e+00, 5.40000000e+01, ...,
        4.98438784e-01, 4.89144317e-01, 4.93636364e-01],
       [1.13794197e+05, 0.00000000e+00, 7.00000000e+01, ...,
        5.01750292e-01, 5.14954486e-01, 5.03714710e-01]]), array([[4.58415980e+04, 7.22977987e+04, 5.30000000e+01, ...,
        4.96711608e-01, 5.19359146e-01, 4.96196606e-01],
       [1.32354614e+05, 0.00000000e+00, 2.3

## AGRAW1 with categories excluded

### Find the best tol and max_iter (the drift type is irrelevant)

In [5]:
from core import mssw_preprocessing
from eval_helpers import kmeans_verbose_helpers


weighted_joined_reference_data, _, _ = mssw_preprocessing.mssw_preprocess(
    agraw1_exclude_reference_batches[abrupt_agraw1_path], agraw1_exclude_testing_batches[abrupt_agraw1_path])

filename = 'agraw1_exclude_new_output.txt'
kmeans_verbose_helpers.write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=100, max_iter=500, tol=0, random_state=1053)
output_dicts = kmeans_verbose_helpers.convert_kmeans_output_file_to_dicts(filename, n_init=100)
kmeans_verbose_helpers.print_stats_from_kmeans_output_dicts(output_dicts)

something
random state: 1053
total number of results: 100
maximum number of iterations: 24
minimum initial inertia: 769.9567074502496
maximum initial inertia: 1491.5073913725237
number of unique final inertia values: 1
minimum final inertia: 568.2804090734008
maximum final inertia: 568.2804090734008
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


### Use them for the analysis

In [6]:
from core import mssw_eval


agraw1_exclude_stats = {}
for agraw1_path in all_agraw1_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        mssw_eval.all_drifting_batches_randomness_robust(
        agraw1_exclude_reference_batches[agraw1_path],
        agraw1_exclude_testing_batches[agraw1_path],
        n_clusters=2,
        n_init=100,
        max_iter=250,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    agraw1_exclude_stats[agraw1_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('AGRAW1 STATS')
print(agraw1_exclude_stats)

min_runs 2
centroid distance sums [[ 998.57287042 1316.87025412]]
num points in clusters [[4145. 5855.]]
centroid distance sums [[1022.38853709 1298.04230858]]
num points in clusters [[4249. 5751.]]
centroid distance sums [[1012.60834134 1299.69090961]]
num points in clusters [[4189. 5811.]]
centroid distance sums [[ 982.73865892 1331.29119407]]
num points in clusters [[4109. 5891.]]
centroid distance sums [[ 998.90425333 1317.39562392]]
num points in clusters [[4139. 5861.]]
centroid distance sums [[1027.33199201 1321.45659182]]
num points in clusters [[4194. 5806.]]
centroid distance sums [[1067.3316959  1309.38181483]]
num points in clusters [[4328. 5672.]]
centroid distance sums [[1063.37344088 1309.54802278]]
num points in clusters [[4317. 5683.]]
centroid distance sums [[1043.11554237 1327.72846044]]
num points in clusters [[4229. 5771.]]
centroid distance sums [[1051.40648462 1324.18529302]]
num points in clusters [[4253. 5747.]]
centroid distance sums [[1316.87025412  998.57287

centroid distance sums [[1316.87025412  998.57287042]]
num points in clusters [[5855. 4145.]]
centroid distance sums [[1298.04230858 1022.38853709]]
num points in clusters [[5751. 4249.]]
centroid distance sums [[1299.69090961 1012.60834134]]
num points in clusters [[5811. 4189.]]
centroid distance sums [[1330.84926747  983.20946113]]
num points in clusters [[5889. 4111.]]
centroid distance sums [[1317.62301075  998.94743326]]
num points in clusters [[5862. 4138.]]
centroid distance sums [[1324.22712614 1004.25235083]]
num points in clusters [[5861. 4139.]]
centroid distance sums [[1312.99583005 1047.49136066]]
num points in clusters [[5731. 4269.]]
centroid distance sums [[1298.46578398 1079.22686534]]
num points in clusters [[5623. 4377.]]
centroid distance sums [[1332.19882539 1045.19453179]]
num points in clusters [[5774. 4226.]]
centroid distance sums [[1333.54605906 1033.24613635]]
num points in clusters [[5805. 4195.]]
min_runs 2
centroid distance sums [[ 998.57287042 1316.87025

### Save the results

In [21]:
from eval_helpers import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in agraw1_exclude_stats.items():
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'exclude'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(250)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('agraw1_exclude_jupyter.csv', index=False)

## AGRAW1 with categories onehot encoded

### Find the best tol and max_iter (the drift type is irrelevant)

In [7]:
from core import mssw_preprocessing
from eval_helpers import kmeans_verbose_helpers


weighted_joined_reference_data, _, _ = mssw_preprocessing.mssw_preprocess(
    agraw1_onehot_reference_batches[abrupt_agraw1_path], agraw1_onehot_testing_batches[abrupt_agraw1_path])

filename = 'agraw1_onehot_new_output.txt'
kmeans_verbose_helpers.write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=100, max_iter=500, tol=0, random_state=1053)
output_dicts = kmeans_verbose_helpers.convert_kmeans_output_file_to_dicts(filename, n_init=100)
kmeans_verbose_helpers.print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 19
minimum initial inertia: 1323.2543959360253
maximum initial inertia: 1451.5853803812658
number of unique final inertia values: 18
minimum final inertia: 728.4372189121426
maximum final inertia: 747.0839411450996
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


### Use them for the analysis

In [8]:
from core import mssw_eval


agraw1_onehot_stats = {}
for agraw1_path in all_agraw1_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        mssw_eval.all_drifting_batches_randomness_robust(
        agraw1_onehot_reference_batches[agraw1_path],
        agraw1_onehot_testing_batches[agraw1_path],
        n_clusters=2,
        n_init=100,
        max_iter=200,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    agraw1_onehot_stats[agraw1_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('AGRAW1 STATS')
print(agraw1_onehot_stats)

min_runs 2
centroid distance sums [[2196.07134528  497.16515425]]
num points in clusters [[7934. 2066.]]
centroid distance sums [[2223.43179319  474.12287029]]
num points in clusters [[8028. 1972.]]
centroid distance sums [[2225.79727851  471.8786353 ]]
num points in clusters [[8039. 1961.]]
centroid distance sums [[2227.78457068  469.76449142]]
num points in clusters [[8046. 1954.]]
centroid distance sums [[2247.66545131  453.63275754]]
num points in clusters [[8115. 1885.]]
centroid distance sums [[2214.13834089  485.23583002]]
num points in clusters [[7986. 2014.]]
centroid distance sums [[2194.28290066  504.53256323]]
num points in clusters [[7910. 2090.]]
centroid distance sums [[2217.26382041  484.51719705]]
num points in clusters [[7991. 2009.]]
centroid distance sums [[2231.59546324  471.40252566]]
num points in clusters [[8048. 1952.]]
centroid distance sums [[2220.36038538  481.054088  ]]
num points in clusters [[8005. 1995.]]
centroid distance sums [[ 497.16515425 2196.07134

KeyboardInterrupt: 

### Save the results

In [24]:
from eval_helpers import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in agraw1_onehot_stats.items():
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'onehot'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(200)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('agraw1_onehot_jupyter.csv', index=False)

## AGRAW1 with categories target encoded

### Find the best tol and max_iter (the drift type is irrelevant)

In [9]:
from core import mssw_preprocessing
from eval_helpers import kmeans_verbose_helpers


weighted_joined_reference_data, _, _ = mssw_preprocessing.mssw_preprocess(
    agraw1_target_reference_batches[abrupt_agraw1_path], agraw1_target_testing_batches[abrupt_agraw1_path])

filename = 'agraw1_target_new_output.txt'
kmeans_verbose_helpers.write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=100, max_iter=500, tol=0, random_state=1053)
output_dicts = kmeans_verbose_helpers.convert_kmeans_output_file_to_dicts(filename, n_init=100)
kmeans_verbose_helpers.print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 30
minimum initial inertia: 938.6378356982315
maximum initial inertia: 1677.6132171746485
number of unique final inertia values: 13
minimum final inertia: 738.6524992884706
maximum final inertia: 792.4147804590955
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


### Use them for the analysis

In [10]:
from core import mssw_eval


agraw1_target_stats = {}
for agraw1_path in all_agraw1_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        mssw_eval.all_drifting_batches_randomness_robust(
        agraw1_target_reference_batches[agraw1_path],
        agraw1_target_testing_batches[agraw1_path],
        n_clusters=2,
        n_init=100,
        max_iter=300,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    agraw1_target_stats[agraw1_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('AGRAW1 STATS')
print(agraw1_target_stats)

min_runs 2
centroid distance sums [[1542.44196914 1129.01618223]]
num points in clusters [[5856. 4144.]]
centroid distance sums [[1517.38804469 1154.81192252]]
num points in clusters [[5751. 4249.]]
centroid distance sums [[1524.01603593 1145.06770508]]
num points in clusters [[5808. 4192.]]
centroid distance sums [[1560.63490122 1119.19849451]]
num points in clusters [[5892. 4108.]]
centroid distance sums [[1546.80195765 1131.12611695]]
num points in clusters [[5860. 4140.]]
centroid distance sums [[1538.78266884 1147.56511178]]
num points in clusters [[5807. 4193.]]
centroid distance sums [[1508.04465009 1194.18278306]]
num points in clusters [[5666. 4334.]]
centroid distance sums [[1514.42626701 1192.23045466]]
num points in clusters [[5682. 4318.]]
centroid distance sums [[1535.04243953 1167.04396605]]
num points in clusters [[5772. 4228.]]
centroid distance sums [[1534.05300676 1170.9119216 ]]
num points in clusters [[5748. 4252.]]
centroid distance sums [[1542.44196914 1129.01618

KeyboardInterrupt: 

### Save the results

In [11]:
from eval_helpers import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in agraw1_target_stats.items():
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'target'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(300)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('agraw1_target_jupyter.csv', index=False)